## This loading is pretty slow

In [2]:
import networkx as nx
import igraph as ig
import leidenalg

# G = np.loadtxt("carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv", delimiter=',')

In [3]:
%%timeit -r 3 -n 1

graph = nx.read_weighted_edgelist("../carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv", delimiter=",")

G = ig.Graph.from_networkx(graph)

4min 17s ± 240 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


## Largest

In [ ]:
graph = nx.read_weighted_edgelist("../carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv", delimiter=",")

G = ig.Graph.from_networkx(graph)

In [37]:
%%timeit

partition = leidenalg.find_partition(G, leidenalg.ModularityVertexPartition)

59.6 s ± 9.24 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Enron

In [28]:
# !tail -n +2 edgelist-enron-n184-forRB.csv > edgelist-enron-n184-forRB2.csv

In [38]:
enron = ig.Graph.from_networkx(nx.read_weighted_edgelist("../carey-tsg-graphs/edgelist-enron-n184-forRB2.csv", delimiter=","))

In [39]:
%%timeit

partition = leidenalg.find_partition(enron, leidenalg.ModularityVertexPartition)

1.66 ms ± 46.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## n500

In [40]:
n500 = ig.Graph.from_networkx(nx.read_weighted_edgelist("../carey-tsg-graphs/edgelist-n500-T40-forRB2.csv", delimiter=","))

In [41]:
%%timeit

partition = leidenalg.find_partition(n500, leidenalg.ModularityVertexPartition)

38 ms ± 883 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Networkit:ParallelLeiden

## MSR graph - 1.2GB

In [5]:
import networkit as nk

In [6]:
!pwd

/home/ubuntu/prog/leiden-runtime-experiments


In [7]:
%%timeit -r 3

nk.graphio.readGraph("../carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv", nk.Format.EdgeList, separator=',', firstNode=1)

50.9 s ± 17.9 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [29]:
# Not sure why this library wants to know the ID of the first node. It's required arg
G = nk.graphio.readGraph("../carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv", nk.Format.EdgeList, separator=',', firstNode=1)

print(G.numberOfNodes(), "nodes")
print(G.numberOfEdges(), "edges")

32277 nodes
26630547 edges


In [30]:
%%timeit
pl = nk.community.ParallelLeiden(G)

pl.run()

180 ms ± 1.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
partition = pl.getPartition()

In [42]:
help(nk.community.ParallelLeiden)

Help on class ParallelLeiden in module networkit.community:

class ParallelLeiden(CommunityDetector)
 |  ParallelLeiden(G, randomize=True, iterations=3, gamma=1)
 |  
 |  Parallel Leiden Algorithm.
 |  
 |  Parameters
 |  ----------
 |  G : networkit.Graph
 |          A graph.
 |  randomize : bool, optional
 |          Whether to randomize the node order or not. Default: True
 |  iterations : int, optional
 |          Maximum count of Leiden runs. Default: 3
 |  gamma : float, optional
 |          Multi-resolution modularity parameter: 1.0 (standard modularity), 0.0 (one community), 2m (singleton communities). Default: 1.0
 |  
 |  Method resolution order:
 |      ParallelLeiden
 |      CommunityDetector
 |      networkit.base.Algorithm
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __reduce__ = __reduce_cython__(...)
 |  
 |  __setstate__ = __setstate_cython__(...)
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defin

## Enron graph

In [8]:
# import numpy as np

# G = np.loadtxt("carey-tsg-graphs/edgelist-enron-n184-forRB2.csv", delimiter=',', dtype=int)

# Find first node ID
# print(np.min(G[:,0]))
# print(np.max(G[:,1]))

In [31]:
# Firstnode ID has to be correct! 0 for enron
G = nk.graphio.readGraph("../carey-tsg-graphs/edgelist-enron-n184-forRB2.csv", nk.Format.EdgeList, separator=',', firstNode=0)

print(G.numberOfNodes(), "nodes")
print(G.numberOfEdges(), "edges")

184 nodes
2215 edges


In [32]:
%%timeit
pl = nk.community.ParallelLeiden(G)

pl.run()

669 µs ± 7.02 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## n500

In [14]:
# import numpy as np

# G = np.loadtxt("carey-tsg-graphs/edgelist-n500-T40-forRB2.csv", delimiter=',')#, dtype=int)
# Find first node ID
# print(np.min(G[:,0])) # First Node ID
# print(np.max(G[:,1]))

1.0
500.0


In [33]:
G = nk.graphio.readGraph("../carey-tsg-graphs/edgelist-n500-T40-forRB2.csv", nk.Format.EdgeList, separator=',', firstNode=1)

print(G.numberOfNodes(), "nodes")
print(G.numberOfEdges(), "edges")

500 nodes
124548 edges


In [34]:
%%timeit
pl = nk.community.ParallelLeiden(G)

pl.run()

6.35 ms ± 78.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Graph Encoder Embedding

In [1]:
!ls ~/prog/GraphEmd/Main/Python

Benchmarks.ipynb   Encode.py	  Hyperperameters.py  __init__.py  tests
Clustering.py	   Evaluation.py  LDA.py	      __pycache__  utils
DataPreprocess.py  GNN.py	  Run.py	      setup.py


In [2]:
!pwd

/home/ubuntu/prog/leiden-runtime-experiments


#### The below sometimes works and sometimes ModuleNotFound - why?

In [3]:
!pwd

/home/ubuntu/prog/leiden-runtime-experiments


In [4]:
# Only absolute paths work

# absolute_path = '/users/ariel/prog/'
absolute_path = '/home/ubuntu/prog/'
graph_path = '/home/ubuntu/prog/'
# graph_path = '/static/ariel'

# Change this depending on your folder structure

In [5]:
print(absolute_path + 'GraphEmd/Main/Python')

/home/ubuntu/prog/GraphEmd/Main/Python


In [6]:
import sys
sys.path.append(absolute_path + 'GraphEmd/Main/Python')
import Evaluation # GEE's Evaluation.py

## MSR 1.2GB

In [7]:
%%timeit
Evaluation.loadGraph(graph_path + "/carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv", weighted=True, randomY=True)

Loading /home/ubuntu/prog//carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv
Loading /home/ubuntu/prog//carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv
Loading /home/ubuntu/prog//carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv
Loading /home/ubuntu/prog//carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv
Loading /home/ubuntu/prog//carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv
Loading /home/ubuntu/prog//carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv
Loading /home/ubuntu/prog//carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv
Loading /home/ubuntu/prog//carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv
12.7 s ± 21.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
G_edgelist, Y, n = Evaluation.loadGraph(graph_path + "/carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv", weighted=True, randomY=True)

Loading /home/ubuntu/prog//carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv


In [16]:
%%timeit
Z, W = Evaluation.graph_encoder_embed(G_edgelist, Y, n, Correlation = False, Laplacian = False)

537 ms ± 2.85 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Enron

In [17]:
G_edgelist, Y, n = Evaluation.loadGraph(graph_path + "/carey-tsg-graphs/edgelist-enron-n184-forRB2.csv", weighted=True, randomY=True)

Loading /home/ubuntu/prog//carey-tsg-graphs/edgelist-enron-n184-forRB2.csv


In [18]:
%%timeit
Z, W = Evaluation.graph_encoder_embed(G_edgelist, Y, n, Correlation = False, Laplacian = False)

1.44 ms ± 1.26 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [9]:
!ls /static/ariel/carey-tsg-graphs

edgelist-enron-n184-forRB2.csv	    edgelist-n500-T40-forRB2.csv
edgelist-msr-n32277-T24-forRB2.csv


## n500

In [19]:
G_edgelist, Y, n = Evaluation.loadGraph(graph_path + "/carey-tsg-graphs/edgelist-n500-T40-forRB2.csv", weighted=True, randomY=True)

Loading /home/ubuntu/prog//carey-tsg-graphs/edgelist-n500-T40-forRB2.csv


In [20]:
%%timeit
Z, W = Evaluation.graph_encoder_embed(G_edgelist, Y, n, Correlation = False, Laplacian = False)

8.03 ms ± 37.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
